In [ ]:
#MOVIE REVIEW CLASSIFICATION
#JANIT BIDHAN
#JBIDHAN@GMU.EDU

In [ ]:
# Import all the Libraries and dependencies 
import re
import timeit
import unicodedata
from operator import itemgetter
import matplotlib.pylab as plt
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from scipy.sparse import csr_matrix, find
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances, linear_kernel
from sklearn.model_selection import StratifiedKFold, train_test_split
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
# Used for filtering Data set
def fixdata(sentences):
    stemmer = SnowballStemmer('english')
    tokenizer = RegexpTokenizer(r'\w+')
    # lemmatizer = WordNetLemmatizer()
    # iterate over all lines for preprocessing
    for idx, text in enumerate(sentences):
        # break the line into words
        words = re.split('(\d+)', text)
        # remove numbers
        text_without_numbers = [word for word in words if not word.isdigit()]
        # join tokens
        text = " ".join(text_without_numbers)
        # break the line into words
        words = tokenizer.tokenize(text)
        # make tokens lowercase
        words = [word.lower() for word in words if type(word) == str]
        # remove stopwords
        words = [word for word in words if not word in stopwords.words('english')]
        # remove more than 3 repeating chars with 1
        words = [re.sub(r'(\w)\1(\1+)(\1+)', r'\1', word) for word in words]
        # remove all the special chars
        words = [re.sub(r'[^a-zA-Z0-9 ]', '', word) for word in words]
        # remove words have length less than 2
        words = [re.sub(r'\b\w{1,2}\b', '', word) for word in words]
        # converting accented characters to normal characters
        words = [unicodedata.normalize('NFD', word).encode('ascii', 'ignore').decode("utf-8") for word in words]
        # remove stem words
        words = [stemmer.stem(word) for word in words]
        # lemmatize words (slower so stemming)
        # words = [lemmatizer.lemmatize(word) for word in words]
        # join the text back and store it back
        text = " ".join(words)
        # replace back the updated line
        sentences.loc[idx] = text

        
# Read the file and store the data
def readTrainfile(filepath):
    read_data = pd.read_csv(filepath, names=['label', 'sentence'], sep='\t')
    fixdata(read_data['sentence'])
    return read_data


# Read the file and store the data
def readtestfile(filepath):
    read_data = pd.read_csv(filepath, names=['sentence'], sep='\t')
    fixdata(read_data['sentence'])
    return read_data


#Save the file the output file
def saveOutput(filePath,data):
    # writing to .txt
    np.savetxt(filePath, data, fmt='%s')

#Vectorize the sentences    
def vectorizingData(read_data_train,read_data_test):
    # Making Bag of Words by vectorizing
    vectorizer = CountVectorizer(analyzer='word', min_df=2, lowercase=True, stop_words="english",
                                 strip_accents="unicode")
    return {"training_set_X": vectorizer.fit_transform(read_data_train), "test_set_X": vectorizer.transform(read_data_test)}


# Cosine Distance
def find_cosine_distance(a, b):
    return (cosine_distances(a, b)).flatten()


# Euclidean Distances
def find_euclidean_distance(a, b):
    return (euclidean_distances(a, b)).flatten()


# Cosine Distance
def find_linear_kernel_distance(a, b):
    return (linear_kernel(a, b)).flatten()

# Locate the k similar neighbors
def get_neighbors(train_set_X, test_set_X, testIdx, num_neighbors):
        distances = find_linear_kernel_distance(test_set_X[testIdx], train_set_X)
        d=num_neighbors+1;
        return distances.argsort()[:-d:-1]


# kNN Algorithm
def k_nearest_neighbors(train_set_X, test_set_X, train_set_Y, testData, num_neighbors):
    predictions = list()
    for testIdx, row in testData.iterrows():
        print(testIdx," ", row)
        neighbors_idx = get_neighbors(train_set_X, test_set_X, testIdx, num_neighbors)
        prediction = 0
        k_near_labels = itemgetter(*neighbors_idx)(train_set_Y)
        # get a list of labels from the neighbors and sum the list
        labels_list = train_set_Y[neighbors_idx].tolist()
        prediction = 0
        if num_neighbors > 1:
            prediction = max(set(k_near_labels), key=k_near_labels.count)
        else:
            prediction = k_near_labels
        predictions.append(prediction)
    
    return (predictions)

In [ ]:
def evaluate_test_algorithm(trainingData, testData, neighbors):
    mats = vectorizingData(trainingData['sentence'],testData['sentence'])
    sparseMat_Train_X = mats["training_set_X"]
    sparseMat_Test_X = mats["test_set_X"]
    Label_Train_Y = trainingData["label"]
    predicted = k_nearest_neighbors(sparseMat_Train_X, sparseMat_Test_X, Label_Train_Y, testData, neighbors)
    return predicted

In [10]:
# Reading training Data
trainingData = readTrainfile("1629732045_4949543_train_new.txt")
#Reading Test File
testData = readtestfile("1629732045_8755195_test_new.txt")
#Calling the evaluate method to get the accuracy for the selected K value
KNN_Value=39
predicted = evaluate_test_algorithm(trainingData,testData,KNN_Value)
#Printing output to the file


0   sentence    low budget film set one locat valley shield ef...
Name: 0, dtype: object
1   sentence    one minut untold alreadi rip techniqu blair wi...
Name: 1, dtype: object
2   sentence    recent purchas dvd heard like robert carlyl   ...
Name: 2, dtype: object
3   sentence    peopl abil use neuron one eat one breath one t...
Name: 3, dtype: object
4   sentence    said titl review pain say hitch reach zenith h...
Name: 4, dtype: object
5   sentence    name sequel classic roadhous dea agent john sc...
Name: 5, dtype: object
6   sentence    read review tend agre comment one thing would ...
Name: 6, dtype: object
7   sentence    norman wisdom final star vehicl departur previ...
Name: 7, dtype: object
8   sentence    watch movi actual sever time want say thing ma...
Name: 8, dtype: object
9   sentence    love sci  will put lot sci  movi  usual underf...
Name: 9, dtype: object
10   sentence    good famili movi laugh wish much school stuff ...
Name: 10, dtype: object
11   sentence    re

96   sentence    movi call blame script direct stanley donen si...
Name: 96, dtype: object
97   sentence    orphan boy name tom tommi pender work pair sha...
Name: 97, dtype: object
98   sentence    get call sequel film american werewolf absolut...
Name: 98, dtype: object
99   sentence    black water one best australian movi seen mani...
Name: 99, dtype: object
100   sentence    actual look forward movi read number review re...
Name: 100, dtype: object
101   sentence    troubl film like mani film fail script   scrip...
Name: 101, dtype: object
102   sentence    hous parti fan hous parti favorit movi time ho...
Name: 102, dtype: object
103   sentence    okay say movi made laugh hard hurt statement m...
Name: 103, dtype: object
104   sentence     know better watch movi scifi  sorri syfi shif...
Name: 104, dtype: object
105   sentence    say movi hilari must grin face entir time like...
Name: 105, dtype: object
106   sentence    dread rubbish like want father ted green wing ...
Name: 106,

189   sentence    quit possibl one worst film ever conceiv cast ...
Name: 189, dtype: object
190   sentence    younger today  bean work genius three time baf...
Name: 190, dtype: object
191   sentence    happen catch  want watch rememb spin magazin a...
Name: 191, dtype: object
192   sentence    talk  hollywood special make dirti danc still ...
Name: 192, dtype: object
193   sentence    scream low budget horror movi shot video featu...
Name: 193, dtype: object
194   sentence    father insist watch film regret wast time watc...
Name: 194, dtype: object
195   sentence    felt last night episod slow kinda bore time ho...
Name: 195, dtype: object
196   sentence    young man fall love princess  battl save fathe...
Name: 196, dtype: object
197   sentence    haunt piec work ten minut long would sooner pa...
Name: 197, dtype: object
198   sentence     kettl charact high popular controversi film f...
Name: 198, dtype: object
199   sentence    much respect firefight unimpress bore film act...
Na

281   sentence    film strang even silent movi essenti follow ad...
Name: 281, dtype: object
282   sentence    actor bland almost imposs tell apart paulin ka...
Name: 282, dtype: object
283   sentence    mayb day movi special five decad later seem qu...
Name: 283, dtype: object
284   sentence    movi power watch movi suppos work needless say...
Name: 284, dtype: object
285   sentence    believ even wast netflix rental complet piec c...
Name: 285, dtype: object
286   sentence    disappoint sequel swan princess see tri stori ...
Name: 286, dtype: object
287   sentence    saw film day santa  film festiv delight honest...
Name: 287, dtype: object
288   sentence    must say impress probabl expect much movi mayb...
Name: 288, dtype: object
289   sentence    rate movi generous sceneri ponder gaudi act pa...
Name: 289, dtype: object
290   sentence    rare movi encompass process problem solv fanta...
Name: 290, dtype: object
291   sentence    letter  knight went colleg older sister tell h...
Na

371   sentence    nuri bilg ceylan film distant uzak third featu...
Name: 371, dtype: object
372   sentence    tend get furious hear lucio fulci reput direct...
Name: 372, dtype: object
373   sentence    movi irredeem bad noth make worth see effect s...
Name: 373, dtype: object
374   sentence    easi see mani peopl consid mood love wong kar ...
Name: 374, dtype: object
375   sentence    friend work blockbust get free movi rental wee...
Name: 375, dtype: object
376   sentence    love movi shock disappoint never made theater ...
Name: 376, dtype: object
377   sentence    adam appl permiss know look also erm wilbur wa...
Name: 377, dtype: object
378   sentence    tale tight wound christian famili three four m...
Name: 378, dtype: object
379   sentence    probabl greatest war film certain one greatest...
Name: 379, dtype: object
380   sentence    know director intent make sure sky almost alwa...
Name: 380, dtype: object
381   sentence    saw movi brief run art hous cinema found funni...
Na

...

24618   sentence    greta garbo first talk pictur mgm wise chose e...
Name: 24618, dtype: object
24619   sentence    would recommend film anyon search relax fun fi...
Name: 24619, dtype: object
24620   sentence    king vladislav angus scrimm romania vampir vam...
Name: 24620, dtype: object
24621   sentence    afraid dark left impress sever differ screenpl...
Name: 24621, dtype: object
24622   sentence    episod seri one hold closest roddenberri origi...
Name: 24622, dtype: object
24623   sentence    last time check think one comment glad peopl c...
Name: 24623, dtype: object
24624   sentence    fame releas  year born young ever rememb origi...
Name: 24624, dtype: object
24625   sentence     last big screen spin popular itv sitcom earli...
Name: 24625, dtype: object
24626   sentence     high recommend two friend great   similar min...
Name: 24626, dtype: object
24627   sentence    much love stori david copperfield cannot claim...
Name: 24627, dtype: object
24628   sentence    movi pain 

24705   sentence    excel stori wonder act amaz product valu cool ...
Name: 24705, dtype: object
24706   sentence    stori buster mistaken dead shot dan notori cri...
Name: 24706, dtype: object
24707   sentence    fabul cinematographi sergei urusevski help mak...
Name: 24707, dtype: object
24708   sentence    seen movi find hard understand mani peopl seem...
Name: 24708, dtype: object
24709   sentence    harrison ford play sergeant dutch van den broe...
Name: 24709, dtype: object
24710   sentence    familiar produc work movi piec crap   never sa...
Name: 24710, dtype: object
24711   sentence    friend carous local movi rental store look som...
Name: 24711, dtype: object
24712   sentence    number oscar hungri biopic late great singer r...
Name: 24712, dtype: object
24713   sentence    sure dislik film much mayb old male someth tar...
Name: 24713, dtype: object
24714   sentence    must one wors movi ever seen par blair witch a...
Name: 24714, dtype: object
24715   sentence    let prefac

24793   sentence    rachel chuck yoman valeri harper gerald mcrane...
Name: 24793, dtype: object
24794   sentence    martin ritt first film offer except existentia...
Name: 24794, dtype: object
24795   sentence    movi one best club scene good soundtrack like ...
Name: 24795, dtype: object
24796   sentence    person think movi pretti good almost rate like...
Name: 24796, dtype: object
24797   sentence    one funniest film ever saw theater back earli ...
Name: 24797, dtype: object
24798   sentence    excel plot line make movi one classic cult nin...
Name: 24798, dtype: object
24799   sentence    harrow movi move relentless still utter uniqu ...
Name: 24799, dtype: object
24800   sentence    thought movi go total lame base advertis saw t...
Name: 24800, dtype: object
24801   sentence    one would term happi tale titl lead charact ed...
Name: 24801, dtype: object
24802   sentence    year keep mind two year ago four two men motiv...
Name: 24802, dtype: object
24803   sentence    death perf

24884   sentence    like realli shock movi act worst ever seen sto...
Name: 24884, dtype: object
24885   sentence    consid huge horror movi fan one night want ren...
Name: 24885, dtype: object
24886   sentence    movi purport charact studi pervers review gull...
Name: 24886, dtype: object
24887   sentence    pdq bach better much bach speak part letter wr...
Name: 24887, dtype: object
24888   sentence    symbiopsychotaxiplasm take one releas came now...
Name: 24888, dtype: object
24889   sentence    must lot background info left movi fact film g...
Name: 24889, dtype: object
24890   sentence    back power ranger becom huge franchis  execut ...
Name: 24890, dtype: object
24891   sentence    movi horribl believ hunk junk even movi critte...
Name: 24891, dtype: object
24892   sentence    know love movi year old watch wish thing laugh...
Name: 24892, dtype: object
24893   sentence    caught film outfest screen los angel juli roug...
Name: 24893, dtype: object
24894   sentence    post disas

24974   sentence    believ pretti stun accomplish take jolli roger...
Name: 24974, dtype: object
24975   sentence    assum previous post came exec product compani ...
Name: 24975, dtype: object
24976   sentence    titl film long one seem outdat style act pathe...
Name: 24976, dtype: object
24977   sentence    found film dvd discount rack worth camera work...
Name: 24977, dtype: object
24978   sentence    saw dark harbor seattl film festiv film autumn...
Name: 24978, dtype: object
24979   sentence    agre vinc movi pave way goodfella scene pesci ...
Name: 24979, dtype: object
24980   sentence     serious film troubl teen singapor countri muc...
Name: 24980, dtype: object
24981   sentence    made halfway movi point realiz absolut care en...
Name: 24981, dtype: object
24982   sentence    best movi watch year excel detail storylin rem...
Name: 24982, dtype: object
24983   sentence    best movi seen beauti talent natali portman gr...
Name: 24983, dtype: object
24984   sentence      agre wea

In [11]:
saveOutput("output-KNN-MinDf-2 keyword and k39.txt", predicted)

In [ ]:
#OUTPUT IS SAVED IN THE ABOVE NAMED FILE
#THANK YOU